In [58]:
# # Uncomment this for using at Google Colab
# !pip -q install nltk
# !pip -q install torch
# !pip -q install gensim
# !pip -q install wordcloud
# !pip -q install torchvision

# from google.colab import drive
# drive.mount('/content/gdrive')
# path = "gdrive/My Drive/AML/Project/Dataset/"

path = '/home/arpan_aml/Project/dataset/'

import os
import csv
import nltk
import _pickle as pickle
import string
import gensim
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.neural_network import MLPClassifier




import torch
import torchvision
import seaborn as sb
from torch import nn
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset

In [59]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
n_epochs = 100

### Loading the saved model

In [60]:
with open(path+'dataset_lstm.pkl', 'rb') as fp:
    data = pickle.load(fp)
trainX = data['trainX']
trainY = data['trainY']
testX = data['testX']
testY = data['testY']

In [61]:
class customDataset(Dataset):
    def __init__(self, dataX, dataY, transform=None):
        dataX = np.resize(dataX, (dataX.shape[0], 1, dataX.shape[1]))
        self.dataX = dataX
        self.dataY = dataY
        self.transform = transform
        
    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self, index):
        return self.dataX[index], self.dataY[index]

In [62]:
train_data = customDataset(trainX, trainY)
train_loader = DataLoader(train_data, batch_size=64, shuffle=False)

test_data = customDataset(testX, testY)
test_loader = DataLoader(test_data, batch_size=64)

In [63]:
# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv1d(in_channels=1, out_channels=4, kernel_size=3, stride=1)
#         self.conv2 = nn.Conv1d(in_channels=4, out_channels=8, kernel_size=3, stride=1)
#         self.fc1 = nn.Linear(2368, 1000)
#         self.fc2 = nn.Linear(1000, 500)
#         self.fc3 = nn.Linear(500, 150)
#         self.fc4 = nn.Linear(300, 150)
#         self.fc5 = nn.Linear(150, 50)
#         self.fc6 = nn.Linear(50, 2)
#         self.relu = nn.ReLU(True)
#     def forward(self, X):
#         X1, X2 = torch.split(X, 300, dim=2)
#         X1 = self.relu(self.conv1(X1))
#         X1 = self.relu(self.conv2(X1))
#         X2 = self.relu(self.conv1(X2))
#         X2 = self.relu(self.conv2(X2))
        
#         X1 = X1.view(X1.shape[0], -1)
#         X2 = X2.view(X2.shape[0], -1)
        
#         X1 = self.relu(self.fc1(X1))
#         X1 = self.relu(self.fc2(X1))
#         X1 = self.relu(self.fc3(X1))
#         X2 = self.relu(self.fc1(X2))
#         X2 = self.relu(self.fc2(X2))
#         X2 = self.relu(self.fc3(X2))

#         X = torch.cat((X1, X2), dim=1)
        
#         X = self.relu(self.fc4(X))
#         X = self.relu(self.fc5(X))
#         X = self.relu(self.fc6(X))
#         return X

In [64]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=4, kernel_size=3, stride=1)
        self.fc1 = nn.Linear(1192, 600)
        self.fc2 = nn.Linear(600, 300)
        self.fc3 = nn.Linear(300, 150)
        self.fc4 = nn.Linear(300, 150)
        self.fc5 = nn.Linear(150, 50)
        self.fc6 = nn.Linear(50, 2)
        self.relu = nn.ReLU(True)
    def forward(self, X):
        X1, X2 = torch.split(X, 300, dim=2)
        X1 = self.relu(self.conv1(X1))
        X2 = self.relu(self.conv1(X2))
        
        X1 = X1.view(X1.shape[0], -1)
        X2 = X2.view(X2.shape[0], -1)
        
        X1 = self.relu(self.fc1(X1))
        X1 = self.relu(self.fc2(X1))
        X1 = self.relu(self.fc3(X1))
        X2 = self.relu(self.fc1(X2))
        X2 = self.relu(self.fc2(X2))
        X2 = self.relu(self.fc3(X2))

        X = torch.cat((X1, X2), dim=1)
        
        X = self.relu(self.fc4(X))
        X = self.relu(self.fc5(X))
        X = self.relu(self.fc6(X))
        return X

In [65]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc = nn.Sequential(
                nn.Linear(426,300),
                nn.ReLU(True),
                nn.Linear(300,100),
                nn.ReLU(True),
                nn.Linear(100,50),
                nn.ReLU(True),
                nn.Linear(50,25),
                nn.ReLU(True),
                nn.Linear(25,10),
                nn.ReLU(True),
                nn.Linear(10,2))
   
    def forward(self, X):
         return self.fc(X)



In [69]:
def get_accuracy(data_loader, model):
    total = 0
    correct = 0

    for X, Y in data_loader:
        X = Variable(X).to(device)
        Y = Y.to(device)
        outputs = model(X)
#         print(outputs.size())
        _, pred = torch.max(outputs.data, 2)
        
        total += Y.size(0)
        pred = pred.view(pred.size(0),)
#         print(pred.size(),Y.size())
#         break
        correct += int((pred == Y).sum())
    accuracy = (100.0 * correct) / total
    return accuracy

In [70]:
model = NN().to(device).double()

model_data = {}
optimizer = torch.optim.Adam(lr=0.001, params=model.parameters())
criterion = nn.CrossEntropyLoss()

train_loss = []
train_accu = []
test_loss = []
test_accu = []

for epoch in range(n_epochs):
    for X, Y in train_loader:
        X = Variable(X).to(device)
        Y = Variable(Y).to(device)
        pred = model(X)
        pred = pred.view(pred.shape[0],-1)
        
#         print(pred.shape, ' ', Y.shape)
        loss = criterion(pred, Y)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss.append(loss.data[0])
    train_accu.append(get_accuracy(train_loader, model))
    test_accu.append(get_accuracy(test_loader, model))
    print('epoch [{}/{}], loss:{:.10f}, Accuracy:{:.10f}'
          .format(epoch + 1, n_epochs, loss.data[0], test_accu[-1]))

    for X, Y in test_loader:
        X = Variable(X).to(device)
        Y = Variable(Y).to(device)
        pred = model(X)
        pred = pred.view(pred.shape[0],-1)
        loss = criterion(pred, Y)
        break
    test_loss.append(loss.data[0])
model_data['train_accu'] = train_accu
model_data['train_loss'] = train_loss
model_data['test_loss'] = test_loss
model_data['test_accu'] = test_accu

torch.save(mdoel, path+'model_deep_network.pt')
with open(path + 'model_data.pkl', 'wb') as fp:
    pickle.dump(model_data, fp)

/home/arpan_aml/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/arpan_aml/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/arpan_aml/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch [1/100], loss:0.4175333709, Accuracy:69.4475000000
epoch [2/100], loss:0.2587286095, Accuracy:69.6550000000
epoch [3/100], loss:0.2956409342, Accuracy:70.3200000000
epoch [4/100], loss:0.2534906585, Accuracy:70.0450000000
epoch [5/100], loss:0.2102061744, Accuracy:70.4875000000
epoch [6/100], loss:0.1863629899, Accuracy:70.7825000000
epoch [7/100], loss:0.2104191242, Accuracy:70.8025000000
epoch [8/100], loss:0.1678928161, Accuracy:70.9825000000
epoch [9/100], loss:0.2166615519, Accuracy:71.1300000000
epoch [10/100], loss:0.1604519437, Accuracy:70.9725000000
epoch [11/100], loss:0.1933946103, Accuracy:71.3600000000
epoch [12/100], loss:0.1474352592, Accuracy:70.9300000000
epoch [13/100], loss:0.1250012053, Accuracy:71.4300000000
epoch [14/100], loss:0.1389182103, Accuracy:71.2850000000
epoch [15/100], loss:0.1701091322, Accuracy:71.2125000000
epoch [16/100], loss:0.1353060086, Accuracy:71.2400000000
epoch [17/100], loss:0.1346511712, Accuracy:71.4400000000
epoch [18/100], loss:0.

NameError: name 'mdoel' is not defined

In [ ]:
get_accuracy(train_loader, model)

In [48]:
for X, Y in train_loader:
    X = Variable(X).to(device)
    print((X))
    break

tensor([[[    0.,     0.,     0.,  ...,  2827.,   328.,  2408.]],

        [[    0.,     0.,     0.,  ...,     8.,    11.,    41.]],

        [[    0.,     0.,     0.,  ...,   225., 29647.,  2023.]],

        ...,

        [[    0.,     0.,     0.,  ...,   376.,  2769.,     8.]],

        [[    0.,     0.,     0.,  ...,   175.,  4062.,  3622.]],

        [[    0.,     0.,     0.,  ...,  3361.,     3.,   154.]]],
       device='cuda:0', dtype=torch.float64)
